In [1]:
import cv2
import csv
import pandas as pd
import glob
import numpy as np
import sys
import starModel as sm
from argparse import ArgumentParser
from node import Node, list_to_nodes, nodes_distance
from tracks import tracks_from_points, get_marks_from_track, draw_circles_on_images_plt
from trackingTool import export_csv

In [2]:
path = "F:/Forth_year/Final-Project/cell_death/data/test-sytox-green/images/images_png"
project_name = "test-sytox-green"
output_path = "F:/Forth_year/Final-Project/cell_death/data/test-sytox-green"
print("Reading images...")
pictures_path = glob.glob(path+'/*.png')
print("Sorting images...")
pictures_path.sort(key=lambda path: int(path.split('_')[len(path.split('_')) - 1][:-4]))
images = []
print("Converting images...")
for p in pictures_path:
    images.append(np.array(cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)))
df = pd.DataFrame({"path": pictures_path, "image": images})
print("initializing Star Model...")
star = sm.StarModel()
print("Calculating points...")
points = star.get_images_points(df['image'].values)
print("Calculating tracks...")
tracks = tracks_from_points(points)
print("Exporting results...")
if export_csv(df, tracks, output_path+f"/{project_name}.csv", len(images)):
    print('OK')
else:
    raise Exception("Failed to export results")

Reading images...
Sorting images...
Converting images...
initializing Star Model...
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Calculating points...
Calculating tracks...
Exporting results...
OK


In [3]:
tracks[1]

(528, 46)

In [9]:
def get_track_details(track, track_id):
    last_node = track
    next_node = track.next
    start_point = track.data
    
    track_prob_avg = track.prob
    track_delta_avg = 0
    track_nodes_count = 1
    
    while(next_node is not None):
        track_delta_avg += nodes_distance(next_node, last_node)
        track_prob_avg += next_node.prob
        track_nodes_count += 1
        next_node = next_node.next
    json_track = {'track_id': track_id,
                  'start_node': start_point,
                  'avg_delta': track_delta_avg / track_nodes_count,
                  'avg_prob': track_prob_avg / track_nodes_count,
                  'node_count': track_nodes_count}
    return json_track

In [11]:
def get_multi_tracks_details(tracks):
    tracks_json = []
    for i in range(len(tracks)):
        tracks_json.append(get_track_details(tracks[i], i))
    return tracks_json

In [12]:
get_multi_tracks_details(tracks)

[{'track_id': 0,
  'start_node': array([538,  58], dtype=int64),
  'avg_delta': 9.215540802074674,
  'avg_prob': 0.7351220607757568,
  'node_count': 20},
 {'track_id': 1,
  'start_node': array([528,  46], dtype=int64),
  'avg_delta': 12.099337390908762,
  'avg_prob': 0.6616166591644287,
  'node_count': 20},
 {'track_id': 2,
  'start_node': array([532,  32], dtype=int64),
  'avg_delta': 10.118604503367207,
  'avg_prob': 0.6110741464715255,
  'node_count': 19},
 {'track_id': 3,
  'start_node': array([536,  50], dtype=int64),
  'avg_delta': 13.192636571296834,
  'avg_prob': 0.720497194925944,
  'node_count': 15},
 {'track_id': 4,
  'start_node': array([686, 492], dtype=int64),
  'avg_delta': 10.23010666660222,
  'avg_prob': 0.8406338691711426,
  'node_count': 20},
 {'track_id': 5,
  'start_node': array([684, 234], dtype=int64),
  'avg_delta': 14.907765705760038,
  'avg_prob': 0.8686789512634278,
  'node_count': 20},
 {'track_id': 6,
  'start_node': array([528, 682], dtype=int64),
  'avg_d